In [0]:
# Which disease has a maximum number of claims.
from pyspark.sql import Window
from pyspark.sql.functions import rank, dense_rank,col

In [0]:
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.access.key", "AK")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "SK")


In [0]:
salesman = spark.read.options(header=True, inferSchema=True, delimiter = '|').json("s3://capstone-kanchan/healthcare/claims.json")
salesman = salesman.dropDuplicates()
grouped = salesman.groupBy(salesman.disease_name).count()
grouped_ranked = grouped.withColumn("rank",dense_rank().over(Window.partitionBy().orderBy(col("count").desc())))
grouped_ranked = grouped_ranked.filter(grouped_ranked.rank ==1)


In [0]:
grouped_ranked.select("disease_name","count").write.format("redshift").option("url", "jdbc:redshift://redshift-serverless.amazonaws.com:5439/dev").\
   option("dbtable", "capstone.max_claims_per_disease").\
   option("aws_iam_role", "arn:aws:iam::Redshift_admin").\
   option("tempdir", "s3a://capstone-kanchan/temp/").\
   option("user", "admin").\
   option("password", "PW").save()